In [1]:
# %% Reading in and formatting individual stats
import pandas as pd

df_bog = pd.read_csv('raw_2017/bogey_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS', 'ROUNDS PLAYED'])
df_bog = df_bog.rename(index=str, columns={'TOTAL BOGEYS': 'B', 'ROUNDS PLAYED': 'Rounds'})

df_db = pd.read_csv('raw_2017/doublebogey_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL BOGEYS AND WORSE'])
df_db = df_db.rename(index=str, columns={'TOTAL BOGEYS AND WORSE': 'DBoW'})

df_bir3 = pd.read_csv('raw_2017/par3_bird_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 3 BIRDIES OR BETTER', 'PAR 3 HOLES'])
df_bir3 = df_bir3.rename(index=str, columns={'PAR 3 BIRDIES OR BETTER': 'P3b', 'PAR 3 HOLES': 'P3H'})

df_eag4 = pd.read_csv('raw_2017/par4_eag_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL'])
df_eag4 = df_eag4.rename(index=str, columns={'TOTAL': 'P4e'})

df_bir4 = pd.read_csv('raw_2017/par4_bird_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 4 BIRDIES OR BETTER', 'PAR 4 HOLES'])
if df_bir4['PAR 4 HOLES'].dtype == 'O':  # need this if the number includes commas and quotes
    df_bir4['PAR 4 HOLES'] = df_bir4['PAR 4 HOLES'].str.replace(',', '').astype(float)
df_bir4 = df_bir4.rename(index=str, columns={'PAR 4 BIRDIES OR BETTER': 'P4be', 'PAR 4 HOLES': 'P4H'})

df_eag5 = pd.read_csv('raw_2017/par5_eag_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'TOTAL'])
df_eag5 = df_eag5.rename(index=str, columns={'TOTAL': 'P5e'})

df_bir5 = pd.read_csv('raw_2017/par5_bird_2017.csv', header=0, index_col='PLAYER NAME', usecols=['PLAYER NAME', 'PAR 5 BIRDIES OR BETTER', 'PAR 5 HOLES'])
df_bir5 = df_bir5.rename(index=str, columns={'PAR 5 BIRDIES OR BETTER': 'P5be', 'PAR 5 HOLES': 'P5H'})

# %% Joining to create overall dataframe
# Now I join them all
df = df_bog.join(df_db).join(df_bir3).join(df_eag4).join(df_bir4).join(df_eag5).join(df_bir5)
df = df.fillna(0)  # this is just for eagle

# Now subtract eagles from birdies or better to get just birdies
df['P4b'] = df['P4be'].subtract(df['P4e'])
df = df.drop('P4be', axis=1)

df['P5b'] = df['P5be'].subtract(df['P5e'])
df = df.drop('P5be', axis=1)

# Now subtract bogeys from bogeys or worse to get double bogeys (or worse)
df['DB'] = df['DBoW'].subtract(df['B'])
df = df.drop('DBoW', axis=1)

In [2]:
df.head()

,B,Rounds,P3b,P3H,P4e,P4H,P5e,P5H,P4b,P5b,DB
PLAYER NAME,,,,,,,,,,,
Jordan Spieth,160,78,52,309,2.0,867.0,6.0,228,189.0,109.0,21
Paul Casey,190,90,49,360,3.0,980.0,7.0,280,184.0,115.0,22
Chad Campbell,211,97,48,392,1.0,1062.0,6.0,292,160.0,118.0,17
Dustin Johnson,154,70,43,277,1.0,763.0,8.0,220,136.0,100.0,16
Chez Reavie,209,95,66,385,0.0,1027.0,8.0,298,178.0,106.0,26


In [8]:
import numpy as np

In [65]:
df2 = df.copy()

In [66]:
df2['P3b'] = df2['P3b'] + (np.sum(df['P3b'])/np.sum(df['P3H'])) * df['P3H'].median() / 6
df2['P3H'] = df2['P3H'] + df['P3H'].median() / 6

In [67]:
df2['P4b'] = df2['P4b'] + (np.sum(df['P4b'])/np.sum(df['P4H'])) * df['P4H'].median() / 6
df2['P4e'] = df2['P4e'] + (np.sum(df['P4e'])/np.sum(df['P4H'])) * df['P4H'].median() / 6
df2['P4H'] = df2['P4H'] + df['P4H'].median() / 6

In [68]:
df2['P5b'] = df2['P5b'] + (np.sum(df['P5b'])/np.sum(df['P5H'])) * df['P5H'].median() / 6
df2['P5e'] = df2['P5e'] + (np.sum(df['P5e'])/np.sum(df['P5H'])) * df['P5H'].median() / 6
df2['P5H'] = df2['P5H'] + df['P5H'].median() / 6

In [79]:
df2['B'] = df2['B'] + (np.sum(df['B'])/np.sum(df['Rounds'])) * df['Rounds'].median() / 6
df2['DB'] = df2['DB'] + (np.sum(df['DB'])/np.sum(df['Rounds'])) * df['Rounds'].median() / 6
df2['Rounds'] = df2['Rounds'] + df['Rounds'].median() / 6

In [80]:
df2.head()

,B,Rounds,P3b,P3H,P4e,P4H,P5e,P5H,P4b,P5b,DB
PLAYER NAME,,,,,,,,,,,
Jordan Spieth,193.612275,90.666667,58.906861,360.166667,2.171539,1005.333333,6.873229,267.666667,211.637023,124.785165,25.363998
Paul Casey,223.612275,102.666667,55.906861,411.166667,3.171539,1118.333333,7.873229,319.666667,206.637023,130.785165,26.363998
Chad Campbell,244.612275,109.666667,54.906861,443.166667,1.171539,1200.333333,6.873229,331.666667,182.637023,133.785165,21.363998
Dustin Johnson,187.612275,82.666667,49.906861,328.166667,1.171539,901.333333,8.873229,259.666667,158.637023,115.785165,20.363998
Chez Reavie,242.612275,107.666667,72.906861,436.166667,0.171539,1165.333333,8.873229,337.666667,200.637023,121.785165,30.363998


In [52]:
df['P5H'].median() / 6

39.666666666666664

In [71]:
np.median(df['P5b']/df['P5H'])

0.39414928292046936

In [54]:
np.sum(df['P4b'])/np.sum(df['P4H'])

0.16364112856832388

In [81]:
aa = df['B']/df['Rounds']

In [82]:
bb = df2['B']/df2['Rounds']

In [83]:
aa.describe()

count    190.000000
mean       2.669209
std        0.262525
min        2.051282
25%        2.495787
50%        2.652248
75%        2.791215
max        3.745455
dtype: float64

In [84]:
bb.describe()

count    190.000000
mean       2.664920
std        0.221373
min        2.135430
25%        2.516923
50%        2.652424
75%        2.768315
max        3.541068
dtype: float64

In [85]:
aa.sort_values()

PLAYER NAME
Jordan Spieth            2.051282
Paul Casey               2.111111
Chad Campbell            2.175258
Dustin Johnson           2.200000
Chez Reavie              2.200000
Patrick Cantlay          2.208333
Webb Simpson             2.232323
Lucas Glover             2.233333
Rickie Fowler            2.236842
Hideki Matsuyama         2.263158
Zac Blair                2.303030
Matt Kuchar              2.315217
Adam Hadwin              2.316327
Charles Howell III       2.317073
Tony Finau               2.317308
Jon Rahm                 2.325301
Sergio Garcia            2.338710
Martin Flores            2.345679
Ian Poulter              2.352941
Justin Thomas            2.367089
Cameron Percy            2.369231
Louis Oosthuizen         2.370968
Pat Perez                2.388889
John Huh                 2.397727
Bill Haas                2.400000
Brian Harman             2.400000
Marc Leishman            2.411111
Tom Hoge                 2.413043
J.T. Poston              2.413793
Br

In [86]:
bb.sort_values()

PLAYER NAME
Jordan Spieth            2.135430
Paul Casey               2.178042
Chad Campbell            2.230507
Chez Reavie              2.253365
Dustin Johnson           2.269503
Webb Simpson             2.280110
Lucas Glover             2.285184
Rickie Fowler            2.296379
Patrick Cantlay          2.301301
Hideki Matsuyama         2.318935
Zac Blair                2.342796
Tony Finau               2.353820
Adam Hadwin              2.354930
Matt Kuchar              2.356168
Charles Howell III       2.362101
Jon Rahm                 2.368769
Martin Flores            2.387320
Sergio Garcia            2.392129
Ian Poulter              2.400152
Justin Thomas            2.406679
Cameron Percy            2.415609
Louis Oosthuizen         2.418914
Pat Perez                2.421548
Brian Harman             2.428511
John Huh                 2.429923
Bill Haas                2.432890
Marc Leishman            2.441029
J.T. Poston              2.444270
Brian Gay                2.455282
Se

In [16]:
df.loc['Rickie Fowler']

B         170.0
Rounds     76.0
P3b        54.0
P3H       304.0
P4e         3.0
P4H       832.0
P5e         3.0
P5H       232.0
P4b       157.0
P5b       114.0
DB         25.0
Name: Rickie Fowler, dtype: float64

In [87]:
df.loc['Tag Ridings']

B         163.0
Rounds     59.0
P3b        40.0
P3H       237.0
P4e         1.0
P4H       636.0
P5e         6.0
P5H       189.0
P4b       109.0
P5b        71.0
DB         15.0
Name: Tag Ridings, dtype: float64

In [43]:
bb = df2['P3b']/df2['P3H']
bb.sort_values()

PLAYER NAME
Si Woo Kim           0.085652
Roberto Castro       0.086088
Chad Collins         0.092356
Hunter Mahan         0.099677
Branden Grace        0.099967
Ryan Moore           0.100518
Rory McIlroy         0.102427
Brandon Hagy         0.102472
Bubba Watson         0.102624
Louis Oosthuizen     0.103310
Ryo Ishikawa         0.103547
Kyle Reifers         0.105379
Steven Bowditch      0.105435
Jimmy Walker         0.106783
Boo Weekley          0.106925
Charl Schwartzel     0.108098
FabiÃ¡n GÃ³mez       0.109147
Rod Pampling         0.109787
Pat Perez            0.110021
SÃ¸ren Kjeldsen      0.111109
Patrick Rodgers      0.111428
Ken Duke             0.112095
Jason Bohn           0.112095
Morgan Hoffmann      0.112471
Ryan Palmer          0.113212
Zach Johnson         0.113894
Xander Schauffele    0.114019
Seamus Power         0.114063
Andrew Loupe         0.114076
Ernie Els            0.114513
                       ...   
Dustin Johnson       0.152078
Matt Every           0.15234

In [48]:
(bb-aa).sort_values()

PLAYER NAME
Tom Hoge            -0.017599
Graeme McDowell     -0.013041
Nicholas Lindheim   -0.011222
Brandt Snedeker     -0.010892
Camilo Villegas     -0.007922
Scott Piercy        -0.007864
Ryan Armour         -0.007498
Rickie Fowler       -0.006143
Tag Ridings         -0.006000
Blayne Barber       -0.005827
Patton Kizzire      -0.005006
Robert Garrigus     -0.004794
Jordan Spieth       -0.004730
Seung-Yul Noh       -0.004574
Luke List           -0.004492
Brett Stegmaier     -0.004448
Cameron Smith       -0.004419
Chez Reavie         -0.004275
Peter Malnati       -0.004049
Matt Jones          -0.003811
Brooks Koepka       -0.003793
Adam Scott          -0.003647
Retief Goosen       -0.003355
J.T. Poston         -0.003353
Steven Alker        -0.003224
Dustin Johnson      -0.003157
Matt Every          -0.003137
Wesley Bryan        -0.003093
J.J. Spaun          -0.003022
Emiliano Grillo     -0.002956
                       ...   
Shawn Stefani        0.003438
Zach Johnson         0.00350